# Softmax exercise

*Complete and hand in this completed worksheet (including its outputs and any supporting code outside of the worksheet) with your assignment submission. For more details see the [assignments page](http://vision.stanford.edu/teaching/cs231n/assignments.html) on the course website.*

This exercise is analogous to the SVM exercise. You will:

- implement a fully-vectorized **loss function** for the Softmax classifier
- implement the fully-vectorized expression for its **analytic gradient**
- **check your implementation** with numerical gradient
- use a validation set to **tune the learning rate and regularization** strength
- **optimize** the loss function with **SGD**
- **visualize** the final learned weights

In [1]:
import os
os.chdir(os.getcwd() + '/..')

# Run some setup code for this notebook
import random
import numpy as np
import matplotlib.pyplot as plt

from utils.data_utils import load_CIFAR10

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# Some more magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [4]:
# Load the raw CIFAR-10 data
cifar10_dir = 'datasets/cifar-10-batches-py'
X_train, y_train, X_test, y_test = load_CIFAR10(cifar10_dir)

# Split the data
num_training = 49000
num_validation = 1000
num_test = 1000
num_dev = 500

mask = range(num_training, num_training+num_validation)
X_val = X_train[mask]
y_val = y_train[mask]

mask = range(num_training)
X_train = X_train[mask]
y_train = y_train[mask]

mask = np.random.choice(num_training, num_dev, replace=False)
X_dev = X_train[mask]
y_dev = y_train[mask]

mask = xrange(num_test)
X_test = X_test[mask]
y_test = y_test[mask]

# Preprocessing: reshape the image data into rows
X_train = X_train.reshape(X_train.shape[0], -1)
X_val = X_val.reshape(X_val.shape[0], -1)
X_test = X_test.reshape(X_test.shape[0], -1)
X_dev = X_dev.reshape(X_dev.shape[0], -1)

# Normalize the data: subtract the mean rows
mean_image = np.mean(X_train, axis=0)
X_train -= mean_image
X_val -= mean_image
X_test -= mean_image
X_dev -= mean_image

# append the bias dimension of ones
X_train = np.hstack((X_train, np.ones((X_train.shape[0], 1))))
X_val = np.hstack((X_val, np.ones((X_val.shape[0], 1))))
X_test = np.hstack((X_test, np.ones((X_test.shape[0], 1))))
X_dev= np.hstack((X_dev, np.ones((X_dev.shape[0], 1))))

print(X_train.shape, X_val.shape, X_test.shape, X_dev.shape)

((49000, 3073), (1000, 3073), (1000, 3073), (500, 3073))


## Softmax Classifier

In [17]:
# softmax_loss_vectorized function
from classifiers.linear_classifier import softmax_loss_vectorized
import time

W = np.random.randn(3073, 10) * 0.0001
loss, grad = softmax_loss_vectorized(W, X_dev, y_dev, 0.0)

# As a rough sanity check, our loss should be something close to -log(0.1).
print('loss: %f' % loss)
print('sanity check: %f' % (-np.log(0.1)))

loss: 2.386956
sanity check: 2.302585


## Inline Question 1:
Why do we expect our loss to be close to -log(0.1)? Explain briefly.**

**Your answer:** *W = 0*

In [19]:
loss, grad = softmax_loss_vectorized(W, X_dev, y_dev, 0.0)

from utils.gradient_check import grad_check_sparse
f = lambda w: softmax_loss_vectorized(w, X_dev, y_dev, 0.0)[0]
grad_numerical = grad_check_sparse(f, W, grad, 10)

loss, grad = softmax_loss_vectorized(W, X_dev, y_dev, 5e1)
f = lambda w: softmax_loss_vectorized(w, X_dev, y_dev, 5e1)[0]
grad_numerical = grad_check_sparse(f, W, grad, 10)

numerical: 0.736112, analytic: 0.736112, relative error: 4.939844e-08
numerical: 1.359312, analytic: 1.359312, relative error: 2.223424e-08
numerical: 3.672828, analytic: 3.672827, relative error: 2.815123e-08
numerical: 5.150041, analytic: 5.150041, relative error: 1.513723e-08
numerical: 0.439351, analytic: 0.439351, relative error: 5.945770e-08
numerical: 3.621422, analytic: 3.621422, relative error: 2.221157e-08
numerical: 1.943753, analytic: 1.943752, relative error: 5.199429e-08
numerical: 0.220881, analytic: 0.220881, relative error: 4.691754e-07
numerical: -9.092706, analytic: -9.092706, relative error: 4.104540e-09
numerical: 0.378179, analytic: 0.378179, relative error: 3.649779e-08
numerical: -1.764118, analytic: -1.764119, relative error: 2.513974e-08
numerical: 3.271042, analytic: 3.271042, relative error: 1.854753e-08
numerical: -1.781404, analytic: -1.781404, relative error: 3.165128e-08
numerical: -2.152419, analytic: -2.152419, relative error: 5.668960e-09
numerical: 0

In [32]:
# Use the validation set to tune hyperparameters
# accuracy over 0.35 on the validation set.

from classifiers.linear_classifier import Softmax

results = {}
best_val = -1
best_softmax = None
learning_rates = [9e-7, 1e-6, 3e-6, 9e-6, 1e-5]
regularization_strengths = [0, 1e1, 2e1, 3e1]


for lr in learning_rates:
    for reg in regularization_strengths:
        model = Softmax()
        model.train(X_train, y_train, learning_rate=lr, reg=reg, num_iters=5000,
                   batch_size=200, verbose=True)
        y_train_pred = model.predict(X_train)
        train_accuracy = np.mean(y_train == y_train_pred)
        y_val_pred = model.predict(X_val)
        val_accuracy = np.mean(y_val == y_val_pred)
        
        results[(lr, reg)] = (train_accuracy, val_accuracy)
        if val_accuracy > best_val:
            best_val = val_accuracy
            best_softmax = model
            
        print('lr %e reg %e train_accuracy: %f val_accuracy: %f' % (lr, reg, train_accuracy, val_accuracy))
    print
    
for lr, reg in sorted(results):
    train_accuracy, val_accuracy = results[(lr, reg)]
    print('lr %e reg %e train_accuracy: %f val_accuracy: %f' % (lr, reg, train_accuracy, val_accuracy))
    
print
print('best validation accuracy achieved during cross-validation: %f' % best_val)

iteration 0 / 5000: loss 5.890208
iteration 100 / 5000: loss 3.194921
iteration 200 / 5000: loss 2.332129
iteration 300 / 5000: loss 2.300826
iteration 400 / 5000: loss 2.217096
iteration 500 / 5000: loss 2.161462
iteration 600 / 5000: loss 2.104302
iteration 700 / 5000: loss 2.125867
iteration 800 / 5000: loss 2.038327
iteration 900 / 5000: loss 2.169287
iteration 1000 / 5000: loss 2.121470
iteration 1100 / 5000: loss 1.999580
iteration 1200 / 5000: loss 2.106824
iteration 1300 / 5000: loss 1.826786
iteration 1400 / 5000: loss 2.143686
iteration 1500 / 5000: loss 2.039867
iteration 1600 / 5000: loss 1.956495
iteration 1700 / 5000: loss 1.888666
iteration 1800 / 5000: loss 1.908971
iteration 1900 / 5000: loss 1.861310
iteration 2000 / 5000: loss 1.938574
iteration 2100 / 5000: loss 1.838085
iteration 2200 / 5000: loss 1.940508
iteration 2300 / 5000: loss 2.019789
iteration 2400 / 5000: loss 1.789948
iteration 2500 / 5000: loss 1.925156
iteration 2600 / 5000: loss 1.902407
iteration 270

In [33]:
for lr, reg in sorted(results):
    train_accuracy, val_accuracy = results[(lr, reg)]
    print('lr %e reg %e train_accuracy: %f val_accuracy: %f' % (lr, reg, train_accuracy, val_accuracy))
    
print
print('best validation accuracy achieved during cross-validation: %f' % best_val)

lr 9.000000e-07 reg 0.000000e+00 train_accuracy: 0.395347 val_accuracy: 0.354000
lr 9.000000e-07 reg 1.000000e+01 train_accuracy: 0.394551 val_accuracy: 0.355000
lr 9.000000e-07 reg 2.000000e+01 train_accuracy: 0.402776 val_accuracy: 0.390000
lr 9.000000e-07 reg 3.000000e+01 train_accuracy: 0.406041 val_accuracy: 0.384000
lr 1.000000e-06 reg 0.000000e+00 train_accuracy: 0.398714 val_accuracy: 0.391000
lr 1.000000e-06 reg 1.000000e+01 train_accuracy: 0.404694 val_accuracy: 0.376000
lr 1.000000e-06 reg 2.000000e+01 train_accuracy: 0.408184 val_accuracy: 0.386000
lr 1.000000e-06 reg 3.000000e+01 train_accuracy: 0.410796 val_accuracy: 0.380000
lr 3.000000e-06 reg 0.000000e+00 train_accuracy: 0.422204 val_accuracy: 0.367000
lr 3.000000e-06 reg 1.000000e+01 train_accuracy: 0.429755 val_accuracy: 0.393000
lr 3.000000e-06 reg 2.000000e+01 train_accuracy: 0.436898 val_accuracy: 0.398000
lr 3.000000e-06 reg 3.000000e+01 train_accuracy: 0.428061 val_accuracy: 0.409000
lr 9.000000e-06 reg 0.000000

In [23]:
# Evaluate the best softmax on test set
y_test_pred = best_softmax.predict(X_test)
test_accuracy = np.mean(y_test == y_test_pred)
print('softmax on raw pixels final test set accuracy: %f' % test_accuracy)

softmax on raw pixels final test set accuracy: 0.338000


In [24]:
# Visualize the learned weights for each class.
w = best_softmax.W[:-1, :] # STRIP OUT THE BIAS
w = w.reshape(32, 32, 3, 10)
w_min, w_max = np.min(w), np.max(w)
classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
for i in range(10):
    plt.subplot(2, 5, i + 1)
    
    #Rescale the weights to be between 0 and 255
    wing = 255.0 * (w[:, :, :, i].squeeze() - w_min) / (w_max - w_min)
    plt.imshow(wing.astype('uint8'))
    plt.axis('off')
    plt.title(classes[i])

NameError: name 'best_svm' is not defined